# Train models for visual proprioception

Train a regression model for visual proprioception. The input is sensory data (eg. a camera image). This is encoded by a p;predefined sensorprocessing component into a latent representation. What we are training and saving here is a regressor that is mapping the latent representation to the position of the robot (eg. a vector of 6 degrees of freedom).

The specification of this regressor is specified in an experiment of the type "visual_proprioception". Running this notebook will train and save this model.

In [1]:
import sys
sys.path.append("..")
from settings import Config

import pathlib
from pprint import pprint
import matplotlib.pyplot as plt

import numpy as np
import torch
import torch.nn as nn
#import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

torch.manual_seed(1)

from visual_proprioception.visproprio_helper import load_demonstrations_as_proprioception_training, get_visual_proprioception_sp
from visual_proprioception.visproprio_models import VisProprio_SimpleMLPRegression

# Move data to GPU (if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Loading pointer config file: C:\Users\lboloni\.config\BerryPicker\mainsettings.yaml
Loading machine-specific config file: G:\My Drive\LotziStudy\Code\PackageTracking\BerryPicker\settings\settings-LotziYoga.yaml


In [2]:
experiment = "visual_proprioception"
# run = "vp_mpl_conv_vae_1"
run = "vp_mpl_propriotuned_cnn_1"
exp = Config().get_experiment(experiment, run)
pprint(exp)

sp = get_visual_proprioception_sp(exp, device)


Note: no system dependent config file G:\My Drive\LotziStudy\Code\PackageTracking\BerryPicker\settings\experiment-config\LotziYoga\visual_proprioception\vp_mpl_propriotuned_cnn_1_sysdep.yaml,
 that is ok, proceeding.
Configuration for experiment: visual_proprioception/vp_mpl_propriotuned_cnn_1 successfully loaded
{'data_dir': WindowsPath('c:/Users/lboloni/Documents/Code/_TempData/BerryPicker-experiments/visual_proprioception/vp_mpl_propriotuned_cnn_1'),
 'encoding_size': 128,
 'epochs': 100,
 'group_name': 'visual_proprioception',
 'loss': 'MSE',
 'output_size': 6,
 'proprioception_input_file': 'train_inputs.pt',
 'proprioception_mlp_model_file': 'proprioception_mlp.pth',
 'proprioception_target_file': 'train_targets.pt',
 'proprioception_test_input_file': 'test_inputs.pt',
 'proprioception_test_target_file': 'test_targets.pt',
 'proprioception_testing_task': 'proprio_regressor_validation',
 'proprioception_training_task': 'proprio_regressor_training',
 'regressor_hidden_size_1': 64,
 

c:\Users\lboloni\Documents\Code\_VirtualEnvironments\Robot\Robot-venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\lboloni\Documents\Code\_VirtualEnvironments\Robot\Robot-venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Users\lboloni\Documents\Code\_Checkouts\BerryPicker\src\visual_proprioception\..\sensorprocessing\sp_propriotuned_cnn.py:84: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possibl

In [3]:
# Create the regression model 

model = VisProprio_SimpleMLPRegression(exp)
if exp["loss"] == "MSE":
    criterion = nn.MSELoss()
elif exp["loss"] == "L1":
    criterion = nn.L1Loss()
else:
    raise Exception(f'Unknown loss type {exp["loss"]}')

optimizer = optim.Adam(model.parameters(), lr=0.001)

### Load and cache the training data. 
* Iterate through the images and process them into latent encodings. 
* Iterate through the json files describing the robot position
* Save the input and target values into files in the experiment directory. These will act as caches for later runs
* Create the training and validation splits

In [4]:
task = exp["proprioception_training_task"]
proprioception_input_file = pathlib.Path(
    exp["data_dir"], exp["proprioception_input_file"])
proprioception_target_file = pathlib.Path(
    exp["data_dir"], exp["proprioception_target_file"])
tr = load_demonstrations_as_proprioception_training(
    sp, task, proprioception_input_file, proprioception_target_file)
inputs_training = tr["inputs_training"]
targets_training = tr["targets_training"]
inputs_validation = tr["inputs_validation"]
targets_validation = tr["targets_validation"]

# Create DataLoaders for batching
batch_size = 32
train_dataset = TensorDataset(inputs_training, targets_training)
test_dataset = TensorDataset(inputs_validation, targets_validation)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Cameras found: ['dev2']
There are 547 steps in this demonstration
This demonstration was recorded by the following cameras: ['dev2']
{'actiontype': 'rc-position-target',
 'camera': 'dev2',
 'cameras': ['dev2'],
 'maxsteps': 547,
 'sensorprocessor': <sensorprocessing.sp_propriotuned_cnn.VGG19ProprioTunedSensorProcessing object at 0x0000018EA79F7090>,
 'source_dir': WindowsPath('C:/Users/lboloni/Documents/Code/_TempData/BerryPicker-demos/demos/proprio_regressor_training/2024_12_26__16_45_02'),
 'trim_from': 1,
 'trim_to': 547}
sensor readings shape torch.Size([1, 3, 256, 256])
sensor readings shape torch.Size([1, 3, 256, 256])
sensor readings shape torch.Size([1, 3, 256, 256])
sensor readings shape torch.Size([1, 3, 256, 256])
sensor readings shape torch.Size([1, 3, 256, 256])
sensor readings shape torch.Size([1, 3, 256, 256])
sensor readings shape torch.Size([1, 3, 256, 256])
sensor readings shape torch.Size([1, 3, 256, 256])
sensor readings shape torch.Size([1, 3, 256, 256])
sensor rea

### Perform the training

In [5]:
def train_and_save_proprioception_model(exp):
    """Trains and saves the proprioception model
    """
    modelfile = pathlib.Path(exp["data_dir"], 
                         exp["proprioception_mlp_model_file"])
    if modelfile.exists():
        raise Exception(f'Model already trained {modelfile}.')
    num_epochs = exp["epochs"]    
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for batch_X, batch_y in train_loader:
            # Forward pass
            predictions = model(batch_X)
            loss = criterion(predictions, batch_y)
            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss / len(train_loader):.4f}')

    # Evaluate the model
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for batch_X, batch_y in test_loader:
            predictions = model(batch_X)
            loss = criterion(predictions, batch_y)
            test_loss += loss.item()

    test_loss /= len(test_loader)
    print(f'Test Loss: {test_loss:.4f}')
    torch.save(model.state_dict(), modelfile)

In [6]:
# modelfile = pathlib.Path(Config()["explorations"]["proprioception_mlp_model_file"])

#if modelfile.exists():
#    model.load_state_dict(torch.load(modelfile))
#else:
train_and_save_proprioception_model(exp)

Epoch [10/100], Loss: 0.0421
Epoch [20/100], Loss: 0.0372
Epoch [30/100], Loss: 0.0326
Epoch [40/100], Loss: 0.0286
Epoch [50/100], Loss: 0.0244
Epoch [60/100], Loss: 0.0216
Epoch [70/100], Loss: 0.0189
Epoch [80/100], Loss: 0.0175
Epoch [90/100], Loss: 0.0158
Epoch [100/100], Loss: 0.0128
Test Loss: 0.0172
